# Classification ALZHEIMER MRI | Pytorch

Loading data from google drive

In [6]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


Importing frameworks and libs

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import cv2
import PIL
from PIL import Image
from torchvision import datasets, models, transforms
from torchvision.transforms import v2
import tqdm

Adding name for classes and adding path to the directory with dataset

In [8]:
category_classes = {
    0: "Mild Demented",
    1: "Moderate Demented",
    2: "Non Demented",
    3: "Very Mild Demented",
}

path_to_data = "/content/drive/MyDrive/Alzheimer MRI Disease Classification Dataset/Data"

Loading training data

In [15]:
data = pd.read_parquet(f"{path_to_data}/train-00000-of-00001-c08a401c53fe5312.parquet", engine="pyarrow")
data.head()

,image,label
0,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,2
1,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,0
2,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,3
3,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,3
4,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,2





Loading testing data




In [16]:
test = pd.read_parquet(f"{path_to_data}/test-00000-of-00001-44110b9df98c5585.parquet", engine="pyarrow")
test.head()

,image,label
0,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,3
1,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,0
2,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,2
3,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,3
4,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,0


Decoding image from file

In [17]:
def dict_to_image(img_dict):
  b_string = img_dict['bytes']
  fbuf_img = np.frombuffer(b_string, np.uint8)
  img = cv2.imdecode(fbuf_img, cv2.IMREAD_GRAYSCALE)
  return img

Preproccesing training data

In [18]:
data['img_arr'] = data['image'].apply(dict_to_image)
data.drop('image', axis=1, inplace=True)
data.head()

,label,img_arr
0,2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


Preproccesing testing data

In [19]:
test['img_arr'] = test['image'].apply(dict_to_image)
test.drop('image', axis=1, inplace=True)
test.head()

,label,img_arr
0,3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [20]:
n_classes = len(category_classes) # num of classes

Class for creating comfortable dataset

In [21]:
class ImageDataset(Dataset):
  def __init__(self, dataframe, transform=None):
    self.dataframe = dataframe
    self.transform = transform

  def __len__(self):
    return len(self.dataframe)

  def __getitem__(self, idx):
    image = self.dataframe.iloc[idx]["img_arr"]
    label = self.dataframe.iloc[idx]["label"]

    if self.transform:
      image = image.astype(np.uint8)
      image = self.transform(image)

    image = torch.tensor(image, dtype=torch.float32).unsqueeze(0)
    label = torch.tensor(label, dtype=torch.long)

    return image, label

Class for creating CNN model

In [22]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
    self.pool1 = nn.MaxPool2d(2, 2)
    self.batch_norm = nn.BatchNorm2d(num_features=32)
    self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
    self.pool2 = nn.MaxPool2d(2, 2)
    self.flatten = nn.Flatten()
    self.fc1 = nn.Linear(64 * 32 * 32, 128)
    self.dropout = nn.Dropout(p=0.25)
    self.out = nn.Linear(128, n_classes)

  def forward(self, x):
    x = F.mish(self.conv1(x))
    x = self.pool1(x)
    x = self.batch_norm(x)
    x = F.mish(self.conv2(x))
    x = self.pool2(x)
    x = self.flatten(x)
    leakyReLU = nn.LeakyReLU(0.01)
    x = leakyReLU(self.fc1(x))
    x = self.dropout(x)
    x = self.out(x)
    return x

Image augmentation

In [23]:
transforms = v2.Compose([
    v2.RandomHorizontalFlip(p=0.3),
    v2.RandomVerticalFlip(p=0.3),
    v2.GaussianBlur(kernel_size=3),
    v2.RandomRotation(degrees=(-90, 90)),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.485], std=[0.229])
])

Hyperparameters

In [24]:
learning_rate = 0.001
n_epochs = 10
batch_size = 32

Loading training dataset

In [25]:
train_dataset = ImageDataset(data, transforms)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

Loading testing dataset

In [26]:
test_dataset = ImageDataset(test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Function for training model

In [27]:
def train_model(model, loader, optimizer, n_epochs):
  criterion = nn.CrossEntropyLoss()

  for epoch in range(n_epochs):
    full_losses = 0.0
    for data in tqdm.tqdm(loader):
      input, label = data[0], data[1]

      optimizer.zero_grad()
      output = model(input)
      loss = criterion(output, label)

      full_losses += loss.item()
      loss.backward()
      optimizer.step()

    epoch_loss = full_losses/len(loader)
    print(f"\nEPOCH: {epoch} | LOSS: {epoch_loss}")
  return model

Training & Testing

In [28]:
model = CNN() # create model
optimizer = optim.AdamW(model.parameters(), lr=learning_rate) # loading optimizer
final_model = train_model(model, train_loader, optimizer, n_epochs) # training model

  1%|          | 1/160 [00:04<12:08,  4.58s/it]


KeyboardInterrupt: 

Function for model prediction

In [29]:
def predict_model(model, data_loader):
  model.eval()

  pred_labels, correct_labels = [], []
  with torch.no_grad():
    for input, label in data_loader:
      output = model(input)
      _, pred = torch.max(output, 1)

      pred_labels.extend(pred)
      correct_labels.extend(label)
  return correct_labels, pred_labels

Testing model

In [30]:
from sklearn.metrics import accuracy_score, f1_score

result = predict_model(final_model, test_loader)

acc = accuracy_score(result[0], result[1]) # accuracy: 95.5%
print("Accuracy:", acc)

f1_sc = f1_score(result[0], result[1], average='micro') # f1 score: 95.5%
print("F1-score:", f1_sc)

NameError: name 'final_model' is not defined

Saving model

In [31]:
torch.save(final_model, "mri_classification_model.v1.pth")